### Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('ML').getOrCreate()

In [2]:
spark

In [3]:
import warnings

# Filter all warnings
warnings.filterwarnings('ignore')


In [4]:
# Read the Dataset
training = spark.read.csv("D:\APACHE SPARK\sample1.csv",header=True,inferSchema=True)

In [5]:
training.show()



+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|Yashwanth| 31|        10| 30000|
|  Akshith| 30|         8| 25000|
|   Naresh| 29|         4| 20000|
|  Praveen| 24|         3| 20000|
|   Shashi| 21|         1| 15000|
|     Teja| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [10]:
output=featureassembler.transform(training)

In [11]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|Yashwanth| 31|        10| 30000|         [31.0,10.0]|
|  Akshith| 30|         8| 25000|          [30.0,8.0]|
|   Naresh| 29|         4| 20000|          [29.0,4.0]|
|  Praveen| 24|         3| 20000|          [24.0,3.0]|
|   Shashi| 21|         1| 15000|          [21.0,1.0]|
|     Teja| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [12]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [13]:
finalized_data=output.select("Independent Features","Salary")

In [14]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [15]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [16]:
# Coefficients
regressor.coefficients

DenseVector([-102.53, 1688.6818])

In [17]:
# Intercepts
regressor.intercept

16470.03994673731

In [18]:
# Prediction
pred_results=regressor.evaluate(test_data)

In [19]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [30.0,8.0]| 25000|26903.59520639148|
+--------------------+------+-----------------+



In [20]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1903.595206391481, 3623674.709796625)